In [28]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [29]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [30]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = utils.secrets_api(project.name)

city_secrets = json.loads(secrets.get_secret("city_secrets").value)
CITY = city_secrets["city"]
latitude = city_secrets["latitude"]
longitude = city_secrets["longitude"]

time_secrets = json.loads(secrets.get_secret("time_secrets").value)
earliest_bike_datetime = time_secrets["earliest_bike_datetime"]
last_bike_datetime = time_secrets["last_bike_datetime"]

today = datetime.datetime.now()

2025-01-07 07:29:04,625 INFO: Closing external client and cleaning up certificates.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494
2025-01-07 07:29:05,740 INFO: Initializing external client
2025-01-07 07:29:05,740 INFO: Base URL: https://c.app.hopsworks.ai:443
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [44]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


2025-01-07 07:32:11,568 ERROR: Flight returned unavailable error, with message: failed to connect to all addresses; last error: UNKNOWN: ipv4:3.19.160.248:5005: tcp handshaker shutdown
Traceback (most recent call last):
  File "c:\Users\isakn\miniconda3\envs\SML_Project\lib\site-packages\hsfs\core\arrow_flight_client.py", line 192, in __init__
    self._health_check()
  File "c:\Users\isakn\miniconda3\envs\SML_Project\lib\site-packages\retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "c:\Users\isakn\miniconda3\envs\SML_Project\lib\site-packages\retrying.py", line 266, in call
    raise attempt.get()
  File "c:\Users\isakn\miniconda3\envs\SML_Project\lib\site-packages\retrying.py", line 301, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "c:\Users\isakn\miniconda3\envs\SML_Project\lib\site-packages\six.py", line 724, in reraise
    raise value
  File "c:\Users\isakn\miniconda3\envs\SML_Project\lib\site-package

FeatureStoreException: Could not read data using Hopsworks Feature Query Service. If the issue persists, use read_options={"use_hive": True} instead.

## Clone and pull the repository with the bike data

In [32]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [33]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Loop through the commits and convert the bike data into a dataframe

In [34]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [35]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  68290eccd75b7d4e47320383a23d86e7a43eff1c  -  2025-01-06 23:49:02+00:00
Processing commit:  a7bc71c83506577ff7dc6e6687e852d36651c6c8  -  2025-01-06 22:49:02+00:00
Processing commit:  3397f6bd8f1013ee14a88f3cb6dc46b7c4706db0  -  2025-01-06 21:49:09+00:00
Processing commit:  133f27c2e3b1237915f63add44b13ef61619910b  -  2025-01-06 20:49:03+00:00
Processing commit:  97141c31253eb495340fd2b1fa675c75fbfadf8e  -  2025-01-06 19:49:05+00:00
Processing commit:  d86a5ba24238c2a6d82943004d7998317a6158c1  -  2025-01-06 18:49:48+00:00
Processing commit:  e794fc5ce20ae14539f97f31a3315af922959f42  -  2025-01-06 17:49:18+00:00
Processing commit:  42967854469793e1d901cb67f10c74a619a510af  -  2025-01-06 16:50:06+00:00
Processing commit:  c307dd3308878520b876dde02b330ff3271e624e  -  2025-01-06 15:49:08+00:00
breaking at:  2025-01-06 15:06:11+00:00
{'CLARENDON ROW': [[1, datetime.datetime(2025, 1, 6, 23, 49, 2, tzinfo=<git.objects.util.tzoffset object at 0x00000182460A41C0>)], [2, dateti

In [36]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station
        df_bike_today = pd.concat([df_bike_today, df])
df_bike_today.dropna(inplace=True)
df_bike_today


,num_bikes_available,datetime,station
0,1,2025-01-06 23:49:02+00:00,CLARENDON ROW
1,2,2025-01-06 22:49:02+00:00,CLARENDON ROW
2,3,2025-01-06 21:49:09+00:00,CLARENDON ROW
3,4,2025-01-06 20:49:03+00:00,CLARENDON ROW
4,0,2025-01-06 19:49:05+00:00,CLARENDON ROW
...,...,...,...
4,4,2025-01-06 19:49:05+00:00,HANOVER QUAY EAST
5,3,2025-01-06 18:49:48+00:00,HANOVER QUAY EAST
6,1,2025-01-06 17:49:18+00:00,HANOVER QUAY EAST
7,8,2025-01-06 16:50:06+00:00,HANOVER QUAY EAST


## Fetch the weather data for the same time period

In [41]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-07 00:00:00+00:00,0.90,-4.923738,0.0,0.0,21.986542,27904.794922,0.3,dublin
1,2025-01-07 01:00:00+00:00,0.60,-4.800121,0.0,0.0,19.602652,27904.794922,0.3,dublin
2,2025-01-07 02:00:00+00:00,0.10,-4.916830,0.0,0.0,17.584810,27904.794922,0.3,dublin
3,2025-01-07 03:00:00+00:00,-0.25,-5.127007,0.0,0.0,17.174677,27904.794922,0.3,dublin
4,2025-01-07 04:00:00+00:00,-0.30,-5.163045,0.0,0.0,17.174677,27904.794922,0.3,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-01-16 19:00:00+00:00,4.65,1.585544,0.0,0.0,10.538843,29183.220703,0.0,dublin
236,2025-01-16 20:00:00+00:00,4.65,1.543089,0.0,0.0,10.829959,29183.220703,0.0,dublin
237,2025-01-16 21:00:00+00:00,4.65,1.523146,0.0,0.0,10.966713,29183.220703,0.0,dublin
238,2025-01-16 22:00:00+00:00,4.65,1.474761,0.0,0.0,11.298495,29183.220703,0.0,dublin


## Insert the bike and weather data into Hopsworks

In [38]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 0.00% |          | Rows 0/1026 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: bike_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
http://c.app.hopsworks.ai/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [42]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df)


Uploading Dataframe: 0.00% |          | Rows 0/240 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
http://c.app.hopsworks.ai/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions


## Change the last_bike_datetime secret

In [43]:
if not df_bike_today.empty:
    try:
        secrets.get_secret("time_secrets").delete()
    except:
        pass

    new_last_bike_datetime_raw = pd.Series.max(df_bike_today['datetime'])
    new_last_bike_datetime = new_last_bike_datetime_raw.strftime('%Y-%m-%d %H:%M:%S %Z')

    time_secrets_dict = {
        "earliest_bike_datetime": earliest_bike_datetime,
        "last_bike_datetime": new_last_bike_datetime
    }

    secrets.create_secret("time_secrets", json.dumps(time_secrets_dict))

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
